In [1]:
from google.colab import userdata, drive
import os
drive.mount('/content/drive')
os.environ['KAGGLE_KEY'] = userdata.get('KAGGLE_KEY')
os.environ['KAGGLE_USERNAME'] = userdata.get('KAGGLE_USERNAME')

Mounted at /content/drive


In [2]:
!pip install optuna
!pip install lightning

# !pip install --extra-index-url https://pypi.nvidia.com --upgrade nvidia-dali-cuda120
# !pip install --extra-index-url https://pypi.nvidia.com --upgrade nvidia-dali-tf-plugin-cuda120

!pip install git+https://github.com/kacky355/my_libraries.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.1/380.1 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.0/233.0 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 812.3/812.3 kB 54.7 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (12

In [4]:
import pandas as pd
import numpy as np
import optuna

import os
import glob
import time
import random
import torch

from logger.mylogger import get_my_logger


def set_seeds(seed):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

def set_logger(name):
    now = time.localtime()
    now = time.strftime("%Y-%m-%d-%H-%M-%S", now)
    log_name = f'{name}-{now}.log'
    logger = get_my_logger('/content/drive/MyDrive/BELKA_model/blendinglogs', log_name)
    return logger
set_seeds(2024)
logger = set_logger('blending_v2_try300')


2024-07-08 10:06:28,836 blending_v2_try300-2024-07-08-10-06-28.log:27 get_my_logger [INFO]: logger has made. log_dir:/content/drive/MyDrive/BELKA_model/blendinglogs/logs


In [3]:
val_results=[
    '/content/drive/MyDrive/BELKA_model/kaggle/working/val_results_SeXc_v3.csv',
    '/content/drive/MyDrive/BELKA_model/kaggle/working/val_results_rental_cnn.csv',
    '/content/drive/MyDrive/BELKA_model/kaggle/working/val_results_res.csv'
]

In [5]:
logger.info(f'{val_results}')
val_results = [pd.read_csv(f, index_col=0) for f in val_results]
val_y = pd.read_csv('/content/drive/MyDrive/BELKA_model/kaggle/working/val_y.csv', index_col=0)

2024-07-08 10:06:29,142 blending_v2_try300-2024-07-08-10-06-28.log:1 <cell line: 1> [INFO]: ['/content/drive/MyDrive/BELKA_model/kaggle/working/val_results_SeXc_v3.csv', '/content/drive/MyDrive/BELKA_model/kaggle/working/val_results_rental_cnn.csv', '/content/drive/MyDrive/BELKA_model/kaggle/working/val_results_res.csv']


In [12]:
import numpy as np
from sklearn.metrics import average_precision_score as APS

def objective(trial):
    i = 0
    weights = []
    preds = []
    for j, pred in enumerate(val_results):
        weight = trial.suggest_float(f'w{j}', .0, 1.0)
        weights.append(weight)
        preds.append(pred.iloc[:,i].values)
    weights = [weights[i]/np.sum(weights) for i in range(len(val_results))]
    weighted_preds = [preds[i]*weights[i] for i in range(len(val_results))]
    all_preds = np.sum(weighted_preds, axis=0)
    print('validation APS  CV score =', APS(val_y.iloc[:, i].values, all_preds, average='micro'))
    return APS(val_y.iloc[:, i].values, all_preds, average='micro')

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=300)


[I 2024-07-07 15:54:31,151] A new study created in memory with name: no-name-f43600c2-6df6-458e-9dbd-30d3e16dabb9


validation APS  CV score = 0.629896534075039


[I 2024-07-07 15:54:45,249] Trial 0 finished with value: 0.629896534075039 and parameters: {'w0': 0.9685945303711766, 'w1': 0.6861642879472833, 'w2': 0.28661615216297454}. Best is trial 0 with value: 0.629896534075039.


validation APS  CV score = 0.6314707516966971


[I 2024-07-07 15:54:55,432] Trial 1 finished with value: 0.6314707516966971 and parameters: {'w0': 0.5914369772943562, 'w1': 0.7398368436446491, 'w2': 0.2499633691765336}. Best is trial 1 with value: 0.6314707516966971.


...

[I 2024-07-07 16:32:02,530] Trial 297 finished with value: 0.6327197942782212 and parameters: {'w0': 0.2247316923508953, 'w1': 0.459927417946104, 'w2': 0.3357985025439222}. Best is trial 231 with value: 0.6327527314915999.


validation APS  CV score = 0.6326804174426941


[I 2024-07-07 16:32:09,257] Trial 298 finished with value: 0.6326804174426941 and parameters: {'w0': 0.13712979599020358, 'w1': 0.48288675994288077, 'w2': 0.35937370392780005}. Best is trial 231 with value: 0.6327527314915999.


validation APS  CV score = 0.632702911029693


[I 2024-07-07 16:32:17,410] Trial 299 finished with value: 0.632702911029693 and parameters: {'w0': 0.2720930754393368, 'w1': 0.5276899676364369, 'w2': 0.39616522203191706}. Best is trial 231 with value: 0.6327527314915999.


In [13]:
bind0 = study.best_params
logger.info(f'bind0 best_params:{bind0}')
bind0_weights =[]
for i in range(3):
    bind0_weights.append(bind0[f'w{i}'])
bind0_weights = [bind0_weights[i]/np.sum(bind0_weights) for i in range(3)]
logger.info(f'bind0_weights:{bind0_weights}')


2024-07-07 16:32:17,426 blending_v2_try300-2024-07-07-15-54-01.log:2 <cell line: 2> [INFO]: bind0 best_params:{'w0': 0.22209866887178745, 'w1': 0.5429839066831079, 'w2': 0.4236072600732803}
2024-07-07 16:32:17,430 blending_v2_try300-2024-07-07-15-54-01.log:7 <cell line: 7> [INFO]: bind0_weights:[0.18684324725837087, 0.4567919152738127, 0.3563648374678165]


In [14]:
import numpy as np
from sklearn.metrics import average_precision_score as APS

def objective(trial):
    i = 1
    weights = []
    preds = []
    for j, pred in enumerate(val_results):
        weight = trial.suggest_float(f'w{j}', .0, 1.0)
        weights.append(weight)
        preds.append(pred.iloc[:,i].values)
    weights = [weights[i]/np.sum(weights) for i in range(len(val_results))]
    weighted_preds = [preds[i]*weights[i] for i in range(len(val_results))]
    all_preds = np.sum(weighted_preds, axis=0)
    print('validation APS  CV score =', APS(val_y.iloc[:, i].values, all_preds, average='micro'))
    return APS(val_y.iloc[:, i].values, all_preds, average='micro')

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=300)


[I 2024-07-07 16:32:17,449] A new study created in memory with name: no-name-5104e310-4694-417a-bcbe-a66a0f8bb639


validation APS  CV score = 0.42295130474348197


[I 2024-07-07 16:32:24,577] Trial 0 finished with value: 0.42295130474348197 and parameters: {'w0': 0.09797863958150721, 'w1': 0.09358160744695498, 'w2': 0.4113272292688105}. Best is trial 0 with value: 0.42295130474348197.


validation APS  CV score = 0.4236570627585655


[I 2024-07-07 16:32:33,440] Trial 1 finished with value: 0.4236570627585655 and parameters: {'w0': 0.7304382056821818, 'w1': 0.4264758242228871, 'w2': 0.4019324712537713}. Best is trial 1 with value: 0.4236570627585655.


...

[I 2024-07-07 17:10:33,084] Trial 297 finished with value: 0.4247659482394322 and parameters: {'w0': 0.5030331698490046, 'w1': 0.8895082971953843, 'w2': 0.7375997507589945}. Best is trial 172 with value: 0.4247683209920571.


validation APS  CV score = 0.4247369258098615


[I 2024-07-07 17:10:41,450] Trial 298 finished with value: 0.4247369258098615 and parameters: {'w0': 0.41473437349125875, 'w1': 0.770182432636373, 'w2': 0.753937572223256}. Best is trial 172 with value: 0.4247683209920571.


validation APS  CV score = 0.42476119978426996


[I 2024-07-07 17:10:48,463] Trial 299 finished with value: 0.42476119978426996 and parameters: {'w0': 0.47582180107928784, 'w1': 0.9200529945288629, 'w2': 0.8086909576177438}. Best is trial 172 with value: 0.4247683209920571.


In [15]:
bind1 = study.best_params
logger.info(f'bind1 best_params:{bind1}')
bind1_weights =[]
for i in range(3):
    bind1_weights.append(bind1[f'w{i}'])
bind1_weights = [bind1_weights[i]/np.sum(bind1_weights) for i in range(3)]
logger.info(f'bind1_weights:{bind1_weights}')

2024-07-07 17:10:48,480 blending_v2_try300-2024-07-07-15-54-01.log:2 <cell line: 2> [INFO]: bind1 best_params:{'w0': 0.4858495581939509, 'w1': 0.8954843074820249, 'w2': 0.7410757316935339}
2024-07-07 17:10:48,485 blending_v2_try300-2024-07-07-15-54-01.log:7 <cell line: 7> [INFO]: bind1_weights:[0.228914135516588, 0.42191870437821144, 0.3491671601052006]


In [16]:
import numpy as np
from sklearn.metrics import average_precision_score as APS

def objective(trial):
    i = 2
    weights = []
    preds = []
    for j, pred in enumerate(val_results):
        weight = trial.suggest_float(f'w{j}', .0, 1.0)
        weights.append(weight)
        preds.append(pred.iloc[:,i].values)
    weights = [weights[i]/np.sum(weights) for i in range(len(val_results))]
    weighted_preds = [preds[i]*weights[i] for i in range(len(val_results))]
    all_preds = np.sum(weighted_preds, axis=0)
    print('validation APS  CV score =', APS(val_y.iloc[:, i].values, all_preds, average='micro'))
    return APS(val_y.iloc[:, i].values, all_preds, average='micro')

study2 = optuna.create_study(direction='maximize')
study2.optimize(objective, n_trials=300)


[I 2024-07-07 17:10:48,500] A new study created in memory with name: no-name-094b51bc-ce63-4a01-b14d-25ec8fc99aa4


validation APS  CV score = 0.9175846223684556


[I 2024-07-07 17:10:56,166] Trial 0 finished with value: 0.9175846223684556 and parameters: {'w0': 0.48952631091850585, 'w1': 0.6116027290372842, 'w2': 0.3445628827136501}. Best is trial 0 with value: 0.9175846223684556.


validation APS  CV score = 0.9169324085965014


[I 2024-07-07 17:11:02,214] Trial 1 finished with value: 0.9169324085965014 and parameters: {'w0': 0.6507037272278562, 'w1': 0.4583260395849569, 'w2': 0.4287928566193734}. Best is trial 0 with value: 0.9175846223684556.


...

[I 2024-07-07 17:44:34,811] Trial 297 finished with value: 0.9179837137272051 and parameters: {'w0': 0.3275109786257765, 'w1': 0.9677556716706931, 'w2': 0.6825148385107811}. Best is trial 231 with value: 0.9179937975698915.


validation APS  CV score = 0.9179912860801325


[I 2024-07-07 17:44:41,605] Trial 298 finished with value: 0.9179912860801325 and parameters: {'w0': 0.37908094453740754, 'w1': 0.9192714170471613, 'w2': 0.7300718917335757}. Best is trial 231 with value: 0.9179937975698915.


validation APS  CV score = 0.9177584421063907


[I 2024-07-07 17:44:48,222] Trial 299 finished with value: 0.9177584421063907 and parameters: {'w0': 0.7360459595475487, 'w1': 0.9724986715699453, 'w2': 0.735626762917999}. Best is trial 231 with value: 0.9179937975698915.


In [17]:
bind2 = study.best_params
logger.info(f'bind2 best_params:{bind2}')
bind2_weights =[]
for i in range(3):
    bind2_weights.append(bind2[f'w{i}'])
bind2_weights = [bind2_weights[i]/np.sum(bind2_weights) for i in range(3)]
logger.info(f'bind2_weights:{bind2_weights}')

2024-07-07 17:44:48,239 blending_v2_try300-2024-07-07-15-54-01.log:2 <cell line: 2> [INFO]: bind2 best_params:{'w0': 0.4858495581939509, 'w1': 0.8954843074820249, 'w2': 0.7410757316935339}
2024-07-07 17:44:48,244 blending_v2_try300-2024-07-07-15-54-01.log:7 <cell line: 7> [INFO]: bind2_weights:[0.228914135516588, 0.42191870437821144, 0.3491671601052006]


In [18]:
test_data = pd.read_parquet('/content/drive/MyDrive/BELKA_model/test.parquet')
test_data.set_index('id', inplace=True)
submissions = [
    pd.read_csv('/content/drive/MyDrive/BELKA_model/submission_SeXception_v3_fold0_ep9.csv', index_col=0),
    pd.read_csv('/content/drive/MyDrive/BELKA_model/submission_SX_v2.csv', index_col=0),
    pd.read_csv('/content/drive/MyDrive/BELKA_model/submission_res.csv', index_col=0),
]

names=[
    'SeXc_v3',
    'SeX',
    'res'
    ]
for sub, name in zip(submissions, names):
    test_data[f'model_{name}'] = sub['binds']

In [19]:
test_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1674896 entries, 295246830 to 296921725
Data columns (total 8 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   buildingblock1_smiles  1674896 non-null  object 
 1   buildingblock2_smiles  1674896 non-null  object 
 2   buildingblock3_smiles  1674896 non-null  object 
 3   molecule_smiles        1674896 non-null  object 
 4   protein_name           1674896 non-null  object 
 5   model_SeXc_v3          1674896 non-null  float64
 6   model_SeX              1674896 non-null  float64
 7   model_res              1674896 non-null  float64
dtypes: float64(3), object(5)
memory usage: 115.0+ MB


In [20]:
test_data['binds'] = 0
test_data.loc[test_data['protein_name']=='BRD4','binds'] = np.sum([test_data[test_data['protein_name']=='BRD4'][f'model_{name}'] * bind0_weights[i] for i, name in enumerate(names)],axis=0)
test_data.loc[test_data['protein_name']=='HSA','binds'] = np.sum([test_data[test_data['protein_name']=='HSA'][f'model_{name}'] * bind1_weights[i] for i, name in enumerate(names)],axis=0)
test_data.loc[test_data['protein_name']=='sEH','binds'] = np.sum([test_data[test_data['protein_name']=='sEH'][f'model_{name}'] * bind2_weights[i] for i, name in enumerate(names)],axis=0)


In [21]:
test_data['binds']

id
295246830    1.322926e-05
295246831    7.147909e-04
295246832    1.953529e-06
295246833    1.924829e-06
295246834    3.460843e-04
                 ...     
296921721    1.393293e-05
296921722    4.099139e-07
296921723    8.726485e-07
296921724    4.074886e-05
296921725    2.298015e-08
Name: binds, Length: 1674896, dtype: float64

In [25]:
test_data['binds'].to_csv(f'/content/drive/MyDrive/BELKA_model/submission_blend_02.csv')


In [26]:
pd.read_csv('/content/drive/MyDrive/BELKA_model/submission_blend_02.csv', index_col=0)

,binds
id,
295246830,1.322926e-05
295246831,7.147909e-04
295246832,1.953529e-06
295246833,1.924829e-06
295246834,3.460843e-04
...,...
296921721,1.393293e-05
296921722,4.099139e-07
296921723,8.726485e-07


In [24]:
assert False

AssertionError: 

In [29]:
rng = np.random.default_rng()
i = 0
weights = []
preds = []
for j, pred in enumerate(val_results):
    weight = rng.random()
    weights.append(weight)
    preds.append(pred.values)
weights = [weights[i]/np.sum(weights) for i in range(len(val_results))]
weighted_preds = [preds[i]*weights[i] for i in range(len(val_results))]
all_preds = np.sum(weighted_preds, axis=0)
print('validation APS  CV score =', APS(val_y.values, all_preds, average='micro'))

validation APS  CV score = 0.7282772055779195


In [6]:
import numpy as np
from sklearn.metrics import average_precision_score as APS

def objective(trial):
    i = 0
    weights = []
    preds = []
    for j, pred in enumerate(val_results):
        weight = trial.suggest_float(f'w{j}', .0, 1.0)
        weights.append(weight)
        preds.append(pred.values)
    weights = [weights[i]/np.sum(weights) for i in range(len(val_results))]
    weighted_preds = [preds[i]*weights[i] for i in range(len(val_results))]
    all_preds = np.sum(weighted_preds, axis=0)
    print('validation APS  CV score =', APS(val_y.values, all_preds, average='micro'))
    return APS(val_y.values, all_preds, average='micro')

study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=300)


[I 2024-07-08 10:08:17,909] A new study created in memory with name: no-name-6dce53c9-1b3b-4ca1-81f4-7dc8f4756eaf


validation APS  CV score = 0.7180587891989896


[I 2024-07-08 10:08:52,888] Trial 0 finished with value: 0.7180587891989896 and parameters: {'w0': 0.7062467173755652, 'w1': 0.5826383002384259, 'w2': 0.5005038850321946}. Best is trial 0 with value: 0.7180587891989896.


validation APS  CV score = 0.7238543803568169


[I 2024-07-08 10:09:19,986] Trial 1 finished with value: 0.7238543803568169 and parameters: {'w0': 0.9197566683449735, 'w1': 0.1644617682298145, 'w2': 0.5448541547870281}. Best is trial 1 with value: 0.7238543803568169.


...

[I 2024-07-08 12:09:22,547] Trial 297 finished with value: 0.7264096781600284 and parameters: {'w0': 0.5582424964996592, 'w1': 1.9885919185911823e-05, 'w2': 0.9995229122589513}. Best is trial 279 with value: 0.726410495064949.


validation APS  CV score = 0.7200634126471536


[I 2024-07-08 12:09:46,086] Trial 298 finished with value: 0.7200634126471536 and parameters: {'w0': 0.6289899369010248, 'w1': 0.020813745579538724, 'w2': 0.11885266391744098}. Best is trial 279 with value: 0.726410495064949.


validation APS  CV score = 0.7263807250138742


[I 2024-07-08 12:10:11,304] Trial 299 finished with value: 0.7263807250138742 and parameters: {'w0': 0.5766449231781234, 'w1': 0.020735688413246505, 'w2': 0.996736708534708}. Best is trial 279 with value: 0.726410495064949.


In [7]:
study.best_params

{'w0': 0.5416913007909215,
 'w1': 0.0003403388160951584,
 'w2': 0.9994963320386479}

In [8]:
all_bind = study.best_params
logger.info(f'all_bind_inren best_params:{all_bind}')
all_weights =[]
for i in range(3):
    all_weights.append(all_bind[f'w{i}'])
all_weights = [all_weights[i]/np.sum(all_weights) for i in range(3)]
logger.info(f'all_weights_inren:{all_weights}')

2024-07-08 12:10:11,348 blending_v2_try300-2024-07-08-10-06-28.log:2 <cell line: 2> [INFO]: all_bind_inren best_params:{'w0': 0.5416913007909215, 'w1': 0.0003403388160951584, 'w2': 0.9994963320386479}
2024-07-08 12:10:11,353 blending_v2_try300-2024-07-08-10-06-28.log:7 <cell line: 7> [INFO]: all_weights_inren:[0.3513989436160777, 0.00022078017548512484, 0.6483802762084371]


In [9]:
test_data = pd.read_parquet('/content/drive/MyDrive/BELKA_model/test.parquet')
test_data.set_index('id', inplace=True)
submissions = [
    pd.read_csv('/content/drive/MyDrive/BELKA_model/submission_SeXception_v3_fold0_ep9.csv', index_col=0),
    pd.read_csv('/content/drive/MyDrive/BELKA_model/submission_rental_cnn.csv', index_col=0),
    pd.read_csv('/content/drive/MyDrive/BELKA_model/submission_res.csv', index_col=0),
]

names=[
    'SeXc_v3',
    'rental',
    'res'
    ]
for sub, name in zip(submissions, names):
    test_data[f'model_{name}'] = sub['binds']

In [11]:
test_data['binds'] = np.sum([test_data[f'model_{name}']* all_weights[i] for i, name in enumerate(names)],axis=0)

In [12]:
test_data['binds']

id
295246830    2.344737e-05
295246831    9.149060e-04
295246832    3.577576e-06
295246833    4.601008e-07
295246834    2.239733e-04
                 ...     
296921721    2.142949e-05
296921722    5.946669e-07
296921723    5.334262e-07
296921724    5.152332e-05
296921725    2.538754e-08
Name: binds, Length: 1674896, dtype: float64

In [13]:
test_data['binds'].to_csv(f'/content/drive/MyDrive/BELKA_model/submission_blend_all_inrental.csv')


In [ ]:
assert False

In [ ]:
np.random.seed(42)
rng = np.random.default_rng()
test = pd.DataFrame(rng.random((5,3)))
weight = pd.DataFrame(rng.random((5,3)))
test*weight

In [ ]:

# test_x = rng.integers(1,37, (1024, 142))
# mask = rng.integers(100,142, 1024)
# for i, mk in enumerate(mask):
#     test_x[i][mk:] = 0
# test_x

In [ ]:
test_mask = test_x.copy()
test_mask[test_mask > 0] = 1
test_mask

In [ ]:
import torch
tensor_x = torch.tensor(test_x)
tensor_mask = tensor_x.clone()
tensor_mask[tensor_mask > 0] = 1
tensor_mask

# Optuna weight deciding

In [ ]:
y = rng.integers(0,2, (100,3))
preds = []
for i in range(3):
    pred = rng.random((100,3))
    preds.append(pred)

model = 0
bind = 0
preds[model][:, bind]

In [ ]:
import torch
import torchmetrics

t_preds = torch.tensor(preds[model])
t_y = torch.tensor(y)
t_preds

In [ ]:
from re import A
from torchmetrics.classification import BinaryAveragePrecision as AP, MultilabelAveragePrecision as MAP
from sklearn.metrics import average_precision_score as APS

metrics = AP(thresholds=None)
APs = []
for i in range(3):
    APs.append(metrics(t_preds[:, i], t_y[:, i]))
map = MAP(3,thresholds=None,average='micro')
single_map = MAP(3, thresholds=None, average='none')
tm_smap = single_map(t_preds, t_y)
tm_map = map(t_preds, t_y)
tm_aps = np.mean(APs)
sk_aps = APS(y,preds[model], average='micro')
print(APs,tm_smap, tm_aps, tm_map, sk_aps)

In [ ]:
y

In [ ]:
pd.DataFrame({f'bind{i}': preds[model][:, i] for i in range(3)})

In [ ]:
study.best_params

In [ ]:
study.best_trial

# make submission

In [ ]:
import os
import glob

BASE_DIR = '/content/drive/MyDrive/kaggle/belka'
subs = glob.glob(os.path.join(BASE_DIR,'', '*.csv'))

test = pd.read_parquet(os.path.join(BASE_DIR, 'test.parquet'))

for i, sub in enumerate(subs):
    df = pd.read_csv(sub)
    test[f'model_{i}'] = df['bind']


